In [1]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None

## Load excel file

In [2]:
kbr_data_path = '../../data/raw/KBR/'
kbr_dir_path = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file_path = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data_path, kbr_dir_path, kbr_file_path), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [3]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

In [4]:
chosen_cols = [
    'IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza)',
    'IDENTYFIKACJA OSOBY_Nr. osoby w gosp. dom.',
    'IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    'DANE DO STATYSTYK_środek transportu',
    '23:59:00_środek transportu podgrupa',
    '23:59:00_środek transportu grupa'
]

data_filtered = data[chosen_cols]

data_filtered.columns = [
    'id', 
    'household_person_id',
    'questionnaire_date',
    'age_bracket', 
    'sex',
    'transport_mode',
    'transport_mode_subgroup',
    'transport_mode_group'
]

data_filtered.fillna(0, inplace=True)
# data_filtered.dropna(inplace=True)
# data_filtered.reset_index(drop=True, inplace=True)

In [5]:
data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,transport_mode,transport_mode_subgroup,transport_mode_group
0,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,sam. osobowy pasażer,sam. osobowy pasażer,komunikacja samochodowa
1,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,sam. osobowy pasażer,sam. osobowy pasażer,komunikacja samochodowa
2,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Kobieta,0,0,0
3,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Mężczyzna,0,0,0
4,LOS_67-67-401,0.0,2018-04-26,61 i więcej (emeryci kobiety),Kobieta,0,0,0


In [6]:
age_bracket_map_dict = {
    '6-15 (dzieci)': '6-15',
    '16-19 (młodzież)': '16-19',
    '20-24 (wiek studencki)': '20-24',
    '25-44 (młodsi pracownicy)': '25-44',
    '45-60 (starsi pracownicy kobiety)': '45-60',  # '45-60(K)',
    '45-65 (starsi pracownicy mężczyźni)': '45-65',  # '45-65(M)',
    '61 i więcej (emeryci kobiety)': '61-x',  # '61-(K)',
    '66 i więcej (emeryci mężczyźni)': '66-x'  # '66-(M)'
}
data_filtered.replace({'age_bracket': age_bracket_map_dict}, inplace=True)

sex_map_dict = {
    'Kobieta': 'K',
    'Mężczyzna': 'M'
}
data_filtered.replace({'sex': sex_map_dict}, inplace=True)

data_filtered['age_sex_comb'] = data_filtered['age_bracket'] + '_' + data_filtered['sex']

data_filtered['person_id'] = data_filtered['id'] + data_filtered['questionnaire_date'].astype('str') + data_filtered['household_person_id'].astype('str')

In [17]:
data_filtered = data_filtered[
    data_filtered['transport_mode_group'] == 'komunikacja samochodowa'
]

In [21]:
def add_is_driver(row):
    if row['transport_mode_subgroup'] == 'sam. osobowy kierowca':
        row['is_driver'] = 1
    elif row['transport_mode_subgroup'] == 'sam. osobowy pasażer':
        row['is_driver'] = 0
    else:
        row['is_driver'] = -1
        
    return row

data_filtered = data_filtered.progress_apply(add_is_driver, axis=1)

100%|██████████| 5079/5079 [00:03<00:00, 1293.04it/s]


In [23]:
data_filtered = data_filtered[
    data_filtered['is_driver'] != -1
]

data_filtered['is_driver'].value_counts()

1    3928
0    1015
Name: is_driver, dtype: int64

## Distribution: driver or passenger grouped by combinations of age group + gender

### Prepare data

In [24]:
data_grouped = data_filtered.groupby(['age_sex_comb', 'is_driver'])['is_driver'].count().reset_index(name='count')

data_grouped.shape[0]

22

In [29]:
data_grouped

,age_sex_comb,is_driver,count
0,16-19_K,0,14
1,16-19_K,1,2
2,16-19_M,0,20
3,16-19_M,1,10
4,20-24_K,0,42
5,20-24_K,1,79
6,20-24_M,0,12
7,20-24_M,1,105
8,25-44_K,0,145
9,25-44_K,1,837


In [28]:
data_filtered[
    (data_filtered['age_sex_comb'] == '16-19_K') &
    (data_filtered['is_driver'] == 0)
].shape[0]

14

### Distribution

In [31]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()

drivers_dist = {}

for age_sex_comb in unique_age_sex_comb:
    drivers_dist[age_sex_comb] = {}

    if age_sex_comb in ['6-15_K', '6-15_M']:
        possible_driver = 0
    else:
        possible_driver = 1

    for driver in range(0, possible_driver+1):
        try:
            count = int(data_grouped[
                (data_grouped['is_driver'] == driver) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        drivers_dist[age_sex_comb][driver] = count

In [32]:
drivers_dist

{'16-19_K': {0: 14, 1: 2},
 '16-19_M': {0: 20, 1: 10},
 '20-24_K': {0: 42, 1: 79},
 '20-24_M': {0: 12, 1: 105},
 '25-44_K': {0: 145, 1: 837},
 '25-44_M': {0: 80, 1: 1238},
 '45-60_K': {0: 124, 1: 366},
 '45-65_M': {0: 61, 1: 918},
 '6-15_K': {0: 135},
 '6-15_M': {0: 116},
 '61-x_K': {0: 227, 1: 88},
 '66-x_M': {0: 39, 1: 285}}

### Probability distribution

In [33]:
for age_sex_comb, is_driver in drivers_dist.items():
    is_driver_sum = np.array(list(is_driver.values())).sum()
    if is_driver_sum != 0:
        for i in list(drivers_dist[age_sex_comb].keys()):
            drivers_dist[age_sex_comb][i] = drivers_dist[age_sex_comb][i] / is_driver_sum

In [34]:
drivers_dist

{'16-19_K': {0: 0.875, 1: 0.125},
 '16-19_M': {0: 0.6666666666666666, 1: 0.3333333333333333},
 '20-24_K': {0: 0.34710743801652894, 1: 0.6528925619834711},
 '20-24_M': {0: 0.10256410256410256, 1: 0.8974358974358975},
 '25-44_K': {0: 0.14765784114052954, 1: 0.8523421588594705},
 '25-44_M': {0: 0.06069802731411229, 1: 0.9393019726858877},
 '45-60_K': {0: 0.2530612244897959, 1: 0.746938775510204},
 '45-65_M': {0: 0.06230847803881512, 1: 0.9376915219611849},
 '6-15_K': {0: 1.0},
 '6-15_M': {0: 1.0},
 '61-x_K': {0: 0.7206349206349206, 1: 0.27936507936507937},
 '66-x_M': {0: 0.12037037037037036, 1: 0.8796296296296297}}

### Save probability distribution to .json

In [35]:
out_dir = '../../data/processed/mc/drivers'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [36]:
out_file = 'drivers_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(drivers_dist, f)